In [1]:
import os

import pandas as pd
import polars as pl
import numpy as np

import kaggle_evaluation.jane_street_inference_server

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in `jane_street_gateway` will run in a different container with direct access to the hidden test set and hand off the data timestep by timestep.



Your code will always have access to the published copies of the files.

In [2]:
lags_ : pl.DataFrame | None = None
test_: pl.DataFrame | None = None

# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 1 minute of the batch features being provided.
import xgboost as xgb

loaded_model = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""

    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_, loaded_model, test_
    if lags is not None:
        lags_ = lags

    test_ = test

    combined_data = test.join(lags_, on=["date_id", "time_id", "symbol_id"], how="left")

    # Generate random values between -5 and +5 for the responder_6 column
    random_values = np.random.uniform(-0.49, 0.44, size=combined_data.height)
    
    # Add the responder_6 column to the DataFrame
    combined_data = combined_data.with_columns(
        pl.Series(name="responder_6", values=random_values)
    )


    # Replace this section with your own predictions
    predictions = combined_data.select(
        'row_id', 'responder_6'
    )

    if len(predictions) != len(test):
        print(f"ERROR: prediction dimension mismatch {len(predictions)} != {len(test)}")
        print(f"Combined df shape: {combined_data.shape}")
        print(f"Test df shape: {test.shape}")
        print(f"Lag df shape: {lags_.shape}")
        print(f"Symbols in lag df: {lags_.select(pl.col('symbol_id').unique())}")

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    # Confirm has as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very first `predict` call, which does not have the usual 1 minute response deadline.

In [3]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

In [4]:
import os
if os.path.isfile('submission.parquet'):
    pl_sub = pl.read_parquet('submission.parquet')
    display(pl_sub)

row_id,responder_6
i64,f64
0,0.026087
1,-0.168058
2,0.300606
3,-0.482854
4,0.258046
…,…
34,0.267937
35,0.383962
36,0.106592
